In [1]:
import json
import pandas as pd
import time
import datetime
from collections import defaultdict
from sql_queries import events_table_insert, teams_table_insert, players_table_insert, related_events_table_insert, matches_table_insert
from create_tables import create_database, drop_tables, create_tables
from collections import Counter

In [2]:
def find_common_keys(json_dicts):
    high_level_keys = Counter()
    dict_count = len(json_dicts)
    for d in json_dicts:
        high_level_keys.update(d.keys())
        
    # identify columns appearing in at least half of data dictionaries
    common_keys = [key for key in high_level_keys.keys() if high_level_keys[key] == dict_count]
    
    return common_keys

In [3]:
with open('./data/15946.json', 'r+') as f:
    match_events = json.load(f)

In [4]:
common_keys = find_common_keys(match_events)

In [5]:
common_keys

['id',
 'index',
 'period',
 'timestamp',
 'minute',
 'second',
 'type',
 'possession',
 'possession_team',
 'play_pattern',
 'team']

In [6]:
cur, conn = create_database()

drop_tables(cur, conn)
create_tables(cur, conn)

In [7]:
def def_value():
    return None

In [8]:
for event in match_events:
    events_table_dict = defaultdict(def_value)
    teams_table_dict = defaultdict(def_value)
    players_table_dict = defaultdict(def_value)
    
    events_table_dict["id"] = event["id"]
    events_table_dict["index"] = event["index"]
    events_table_dict["type"] = event["type"]["name"]
    events_table_dict["play_pattern"] = event["play_pattern"]["name"]
    events_table_dict["event_time"] = event["timestamp"]
    events_table_dict["period"] = event["period"]
    events_table_dict["possession"] = event["possession"]
    events_table_dict["possession_team"] = event["possession_team"]["id"]
    events_table_dict["team"] = event["team"]["id"]
    
    teams_table_dict["id"] = event["team"]["id"]
    teams_table_dict["name"] = event["team"]["name"]
    
    players_table_dict["team"] = event["team"]["id"]
    
    
    # TODO: assign the match ID to the events_table_dict and matches_table_dict
    events_table_dict["match"] = 15946
    
    for key, value in event.items():
        if key == "location":
            location_x, location_y = value
            
            events_table_dict["location_x"] = location_x,
            events_table_dict["location_y"] = location_y
            
        elif key == "possession_team":
            events_table_dict["possession_team"] = value["id"]
            
        elif key == "player":
            events_table_dict["player"] = value["id"]
            
            players_table_dict["id"] = value["id"]
            players_table_dict["name"] = value["name"]
            
        elif key == "duration":
            events_table_dict["duration"] = value
            
        elif key == "position":
            players_table_dict["position"] = value["name"]
            
        elif key == "related_events":
            for event_id in value:
                cur.execute(
                    related_events_table_insert,
                    (
                        event["id"],
                        event_id
                    )
                )
            
    cur.execute(
        events_table_insert,
        (
            events_table_dict["id"],
            events_table_dict["index"],
            events_table_dict["type"],
            events_table_dict["play_pattern"],
            events_table_dict["event_time"],
            events_table_dict["period"],
            events_table_dict["location_x"],
            events_table_dict["location_y"],
            events_table_dict["possession_team"],
            events_table_dict["possession"],
            events_table_dict["team"],
            events_table_dict["player"],
            events_table_dict["team"],
            events_table_dict["duration"],
        )
    )
    
    cur.execute(
        teams_table_insert,
        (
            teams_table_dict["id"],
            teams_table_dict["name"],
            teams_table_dict["gender"],
            teams_table_dict["country"]
        )
    )
    
    if players_table_dict["id"]:
        cur.execute(
            players_table_insert,
            (
                players_table_dict["id"],
                players_table_dict["name"],
                players_table_dict["position"],
                players_table_dict["team"]
            )
        )
conn.commit()

In [9]:
with open('../open-data/data/matches/11/1.json', 'r+') as f:
    match_details = json.load(f)

In [10]:
match_keys = find_common_keys(match_details)

In [11]:
for match in match_details:
    match_table_dict = defaultdict(def_value)
    
    match_table_dict["id"] = match["match_id"],
    match_table_dict["stadium"] = match["stadium"]["id"]
    match_table_dict["match_date"] = match["match_date"]
    match_table_dict["competition"] = match["competition"]["competition_id"]
    match_table_dict["home_team"] = match["home_team"]["home_team_id"]
    match_table_dict["away_team"] = match["away_team"]["away_team_id"]
    match_table_dict["home_score"] = match["home_score"]
    match_table_dict["away_score"] = match["away_score"]

    cur.execute(
        matches_table_insert,
        (
            match_table_dict["id"],
            match_table_dict["stadium"],
            match_table_dict["match_date"],
            match_table_dict["competition"],
            match_table_dict["home_team"],
            match_table_dict["away_team"],
            match_table_dict["home_score"],
            match_table_dict["away_score"]
        )
    )
conn.commit()

defaultdict(<function def_value at 0x11e91b430>, {'id': (9799,), 'stadium': 347, 'match_date': '2018-02-17', 'competition': 11, 'home_team': 322, 'away_team': 217, 'home_score': 0, 'away_score': 2})
defaultdict(<function def_value at 0x11e91b430>, {'id': (9636,), 'stadium': 342, 'match_date': '2017-10-01', 'competition': 11, 'home_team': 217, 'away_team': 208, 'home_score': 3, 'away_score': 0})
defaultdict(<function def_value at 0x11e91b430>, {'id': (9609,), 'stadium': 342, 'match_date': '2017-09-19', 'competition': 11, 'home_team': 217, 'away_team': 322, 'home_score': 6, 'away_score': 1})
defaultdict(<function def_value at 0x11e91b430>, {'id': (9575,), 'stadium': 342, 'match_date': '2017-08-20', 'competition': 11, 'home_team': 217, 'away_team': 218, 'home_score': 2, 'away_score': 0})
defaultdict(<function def_value at 0x11e91b430>, {'id': (9928,), 'stadium': 342, 'match_date': '2018-05-09', 'competition': 11, 'home_team': 217, 'away_team': 222, 'home_score': 5, 'away_score': 1})
defau

In [12]:
conn.close()